In [3]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import concurrent.futures

# Carregar o dataset original e selecionar as 100 primeiras linhas
df = pd.read_csv("Results 2.csv")
df_100 = df.head(200)

# Definir as categorias para classificação
categorias = ['PHC', 'Financeiro', 'Service desk', 'Problemas de Ficha', 'Contrato', 'Duvida', 'WinArmazens','Erro no PHC','Password','PHC on','Impressao de Documentos'
              ,'Faturação', 'Phcweb', 'Vencimentos','Integrações','IRS','Fatura Eletronica','ODBC','Acessos','Pedido Urgente'
              ,'desenvolvimento','contato sem sucesso','IDU','Emails','imobilizado','exportação de ficheiros','InformaDB','intalação de phc','campos não preenchidos']

# Carregar o modelo de classificação zero-shot
classificador = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",temperature = 0.1,)

# Concatenar as colunas problema e solução em uma nova coluna para classificação
df_100["concatenação"] = "<pergunta> " + df_100["problema"] + "<fimpergunta> <resposta> " + df_100["solucao"] + "<fimresposta>"

# Adicionar a barra de progresso ao aplicar a classificação
tqdm.pandas()

# classificador.function_to_apply('Sigmoid')

# Função para classificar o âmbito
def classificar_ambito(pergunta):
    resultado = classificador(pergunta, candidate_labels=categorias)
    return resultado['labels'][0]

# Aplicar a classificação com barra de progresso
#df_100["ambito"] = df_100["concatenação"].progress_apply(classificar_ambito)

with concurrent.futures.ThreadPoolExecutor() as executor:
    df_100["ambito"] = list(tqdm(executor.map(classificar_ambito, df_100["concatenação"]), total=len(df_100)))



# Exibir o DataFrame com a nova coluna de âmbito
#print(df_100.head())

df_100.to_excel('classificado4.xlsx')


C:\Users\nando\AppData\Local\Temp\ipykernel_14772\2686554171.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100["concatenação"] = "<pergunta> " + df_100["problema"] + "<fimpergunta> <resposta> " + df_100["solucao"] + "<fimresposta>"
100%|██████████| 200/200 [15:20<00:00,  4.60s/it]
C:\Users\nando\AppData\Local\Temp\ipykernel_14772\2686554171.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100["ambito"] = list(tqdm(executor.map(classificar_ambito, df_100["concatenação"]), total=len(df_100)))